In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain.agents import AgentExecutor
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.agents.structured_chat.base import StructuredChatAgent
from langchain.tools.render import render_text_description
from composio_langchain import ComposioToolSet

In [4]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
tools = composio_toolset.get_tools(actions=['YOUTUBE_SEARCH_YOU_TUBE'])

In [5]:
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")

In [6]:
system_message = """
You are an assistant helping product builders choose the best YouTube channels for product ads or sponsorships. Use the available YouTube search tool to find relevant channels based on the user's input. Provide helpful insights from the top results, such as channel name, content type, and audience fit.

When you use a tool, format your message like this:

<tool>Channel Name</tool>
- Reason for selecting this channel
- Other insights
- Top video of that channel which suggested you this channel
Start by understanding what the user is asking, then use the appropriate tool to gather information on YouTube channels. Once you have the data, analyze and provide recommendations based on factors like relevance to the product, audience, and content style.
"""

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [8]:
agent = StructuredChatAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
task = "machine learning for beginners"
result = agent_executor.invoke({"input": task})
print(result)



> Entering new AgentExecutor chain...
Let me search for some beginner-friendly machine learning videos that can help introduce the topic.

Action:
```{
  "action": "YOUTUBE_SEARCH_YOU_TUBE",
  "action_input": {
    "q": "machine learning for beginners tutorial",
    "maxResults": 5,
    "type": "video"
  }
}
```


Observation: {'data': {'response_data': {'kind': 'youtube#searchListResponse', 'etag': 'eunqnTdbOnLV5j0c6Gzh-M4Qo4E', 'nextPageToken': 'CAUQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#searchResult', 'etag': '3zmqhaMCHDBqEWimvFvvJbZtXow', 'id': {'kind': 'youtube#video', 'videoId': 'ukzFI9rgwfU'}, 'snippet': {'publishedAt': '2018-09-19T14:57:02Z', 'channelId': 'UCsvqVGtbbyHaMoevxPAq9Fg', 'title': 'Machine Learning | What Is Machine Learning? | Introduction To Machine Learning | 2024 | Simplilearn', 'description': 'Artificial Intelligence Engineer (IBM) ...', 'thumbnails': {'default': {'url': 'https://i.ytimg